In [ ]:
# 验证路径设置
import os
import sys

# 自动计算项目根目录
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))  
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
# 打印当前环境路径
print(f"Python executable: {sys.executable}")
print(f"Project root: {project_root}")
# 导入模块
try:
    from config.tickers import NASDAQ_100
    from config.NASDAQ_100_Chinese  import NASDAQ_100_Chinese
    from importnb import Notebook
    from openai import OpenAI 
    from sklearn.model_selection import train_test_split
    with Notebook():
        from src.visualization import plot_technical
        from src.downloader import download_stock_data
    print("✅ 模块导入成功！")
except ImportError as e:
    print(f"❌ 导入失败: {str(e)}")
    print("当前 Python 路径：", sys.path)


Python executable: /opt/anaconda3/envs/quant/bin/python
Project root: /Users/zhengyi/stock_project


[*********************100%***********************]  1 of 1 completed

✅ 模块导入成功！


In [4]:
# 下载股票数据
start_date = '2024-01-01'
end_date = '2025-04-01'
ticker = ['PDD']

df = download_stock_data( ticker , start_date , end_date )
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume,Daily Return,Log Return,Range Pct,MA10,MA50,Volatility,RSI
Date,,,,,,,,,,,,
2024-01-02,143.880005,145.690002,141.899994,145.639999,8211900,NaN,NaN,2.670901,NaN,NaN,NaN,NaN
2024-01-03,143.779999,150.369995,143.619995,148.949997,7504100,2.272726,0.022473,4.699903,NaN,NaN,NaN,NaN
2024-01-04,148.410004,150.190002,147.470001,147.740005,4435900,-0.812347,-0.008157,1.844444,NaN,NaN,NaN,NaN
2024-01-05,147.279999,148.940002,145.809998,146.529999,4394600,-0.819011,-0.008224,2.146633,NaN,NaN,NaN,NaN
2024-01-08,144.009995,147.979996,143.279999,147.550003,6195500,0.696106,0.006937,3.280288,NaN,NaN,NaN,NaN


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [7]:
# 特征工程：创建滞后特征
for lag in [1, 3, 5]:
    df[f'Close_lag_{lag}'] = df['Close'].shift(lag)

In [8]:
# 清理缺失值并准备数据
df_clean = df.dropna()
X = df_clean[['Close_lag_1', 'Close_lag_3', 'Close_lag_5', 'MA50', 'RSI']]
y = df_clean['Close']

In [9]:
# 训练测试分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
# 训练模型
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

RandomForestRegressor()

In [11]:
# 评估
print(f"Test R^2: {model.score(X_test, y_test)}")

Test R^2: 0.9675104449620846
